In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4

from KNN.UserKNNCBF1Recommender import UserKNNCBF1Recommender
from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender


#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [3]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')
UCM_all = scipy.sparse.load_npz('./Matrix/UCM_all_matrix.npz')

#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

In [ ]:
CFrecommender = ItemKNNCFRecommender(URM_train) 
CFrecommender.fit()
SLIMrecommender = SLIM_BPR_Cython(URM_train) 
SLIMrecommender.fit()
CBFrecommender = ItemKNNCBFRecommender(ICM_all,URM_train) 
CBFrecommender.fit()
SLIMErecommender = SLIMElasticNetRecommender(URM_train) 
SLIMErecommender.fit()
UCFRecommender = UserKNNCFRecommender(URM_train)
UCFRecommender.fit()

Similarity column 20635 ( 100 % ), 3355.06 column/sec, elapsed time 0.10 min
SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Processed 1165777 ( 100.00% ) in 7.30 seconds. BPR loss is 9.46E-01. Sample per second: 159766
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.30 min
Processed 1165777 ( 100.00% ) in 5.75 seconds. BPR loss is 2.14E+00. Sample per second: 202726
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.39 min
Processed 1165777 ( 100.00% ) in 5.94 seconds. BPR loss is 2.98E+00. Sample per second: 196277
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.48 min
Processed 1165777 ( 100.00% ) in 14.32 seconds. BPR loss is 3.66E+00. Sample per second: 81431
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.70 min
Processed 1165777 ( 100.00% ) in 8.73 seconds. BPR loss is 4.27E+00. Sample per second: 133503
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.84 min
Processed 1165777 ( 100.00% ) in 7.18 seconds. BPR los

SLIM_BPR_Recommender: Epoch 52 of 100. Elapsed time 4.91 min
Processed 1165777 ( 100.00% ) in 6.84 seconds. BPR loss is 1.49E+01. Sample per second: 170391
SLIM_BPR_Recommender: Epoch 53 of 100. Elapsed time 5.01 min
Processed 1165777 ( 100.00% ) in 6.39 seconds. BPR loss is 1.50E+01. Sample per second: 182343
SLIM_BPR_Recommender: Epoch 54 of 100. Elapsed time 5.10 min
Processed 1165777 ( 100.00% ) in 5.20 seconds. BPR loss is 1.51E+01. Sample per second: 224333
SLIM_BPR_Recommender: Epoch 55 of 100. Elapsed time 5.18 min
Processed 1165777 ( 100.00% ) in 5.02 seconds. BPR loss is 1.52E+01. Sample per second: 232029
SLIM_BPR_Recommender: Epoch 56 of 100. Elapsed time 5.26 min
Processed 1165777 ( 100.00% ) in 5.06 seconds. BPR loss is 1.54E+01. Sample per second: 230619
SLIM_BPR_Recommender: Epoch 57 of 100. Elapsed time 5.35 min
Processed 1165777 ( 100.00% ) in 4.96 seconds. BPR loss is 1.55E+01. Sample per second: 234945
SLIM_BPR_Recommender: Epoch 58 of 100. Elapsed time 5.43 min
Pro

In [ ]:
SuperSimilarity = ItemKNNSimilarityHybridRecommender(URM_train, SLIMrecommender, SLIMErecommender)
SuperSimilarity.fit(alpha=0.5, topK=600)

H3SimilaritySuper = ItemKNNSimilarityHybridRecommender3(URM_train, SuperSimilarity, CFrecommender,CBFrecommender)
H3SimilaritySuper.fit(topK=800, alpha=0.7, beta=0.4, gamma=0.2)

result = H3SimilaritySuper.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

In [21]:
User1CBFrecommender = UserKNNCBF1Recommender(UCM_all,URM_train) 
User1CBFrecommender.fit(topK=200)
CFrecommender = ItemKNNCFRecommender(URM_train)
CFrecommender.fit()

HScores = ItemKNNScoresHybridRecommender(URM_train,CFrecommender,User1CBFrecommender)
alpha =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for a in alpha :
    print(a)
    HScores.fit(alpha=a)
    result = HScores.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Similarity column 27156 ( 54 % ), 800.93 column/sec, elapsed time 0.57 min
Similarity column 50446 ( 100 % ), 786.41 column/sec, elapsed time 1.07 min
Similarity column 20635 ( 100 % ), 3127.96 column/sec, elapsed time 0.11 min
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 14.79 seconds. Users per second: 654
Recommender MAP is= 0.2165486311480633
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 12.43 seconds. Users per second: 778
Recommender MAP is= 0.1819106692904529
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 10.68 seconds. Users per second: 906
Recommender MAP is= 0.15900018117232684
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 10.19 seconds. Users per second: 950
Recommender MAP is= 0.1425092972305204
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 11.62 seconds. Users per second: 833
Recommender MAP is= 0.12771687890147707
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 11.08 seconds. Users per second: 874
Recommender MAP is= 0.1

In [27]:
User1CBFrecommender = UserKNNCBF1Recommender(UCM_all,URM_train) 
User1CBFrecommender.fit(topK=200)

result = User1CBFrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

b=User1CBFrecommender.compute_item_score(all_userID)
print(b.shape)

Similarity column 24987 ( 50 % ), 829.82 column/sec, elapsed time 0.50 min
Similarity column 49788 ( 99 % ), 797.72 column/sec, elapsed time 1.04 min
Similarity column 50446 ( 100 % ), 792.44 column/sec, elapsed time 1.06 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 9.28 seconds. Users per second: 1043
Recommender MAP is= 0.2671809750849438
(50446, 20635)
